In [1]:
import pandas as pd
import numpy as np
import time
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [2]:
def selectkbest(indep_x,dep_y,n):
    test=SelectKBest(score_func=chi2,k=n)
    fit1=test.fit(indep_x,dep_y)
    selectk_features=fit1.transform(indep_x)
    return selectk_features
    
def split_scalar(indep_x,dep_y):
    x_train,x_test,y_train,y_test=train_test_split(indep_x,dep_y,test_size=0.25,random_state=0)
    sc=StandardScaler()
    x_train=sc.fit_transform(x_train)
    x_test=sc.transform(x_test)
    return x_train,x_test,y_train,y_test

def cm_prediction(classifier,x_test):
    y_pred=classifier.predict(x_test)
#Making confusion matrix
    from sklearn.metrics import confusion_matrix
    cm=confusion_matrix(y_test,y_pred)
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    accuracy=accuracy_score(y_test,y_pred)
    report=classification_report(y_test,y_pred)
    return classifier,accuracy,report,x_test,y_test,cm

def logistic(x_train,y_train,x_test):
    from sklearn.linear_model import LogisticRegression
    classifier=LogisticRegression(random_state=0)
    classifier.fit(x_train,y_train)
    classifier,accuracy,report,x_test,y_test,cm=cm_prediction(classifier,x_test)
    return classifier,accuracy,report,x_test,y_test,cm

def svm_linear(x_train,y_train,x_test):
    from sklearn.svm import SVC
    classifier=SVC(kernel="linear",random_state=0)
    classifier.fit(x_train,y_train)
    classifier,accuracy,report,x_test,y_test,cm=cm_prediction(classifier,x_test)
    return classifier,accuracy,report,x_test,y_test,cm

def svm_NL(x_train,y_train,x_test):
    from sklearn.svm import SVC
    classifier=SVC(kernel="rbf",random_state=0)
    classifier.fit(x_train,y_train)
    classifier,accuracy,report,x_test,y_test,cm=cm_prediction(classifier,x_test)
    return classifier,accuracy,report,x_test,y_test,cm

def Navie(x_train,y_train,x_test):
    from sklearn.naive_bayes import GaussianNB
    classifier=GaussianNB()
    classifier.fit(x_train,y_train)
    classifier,accuracy,report,x_test,y_test,cm=cm_prediction(classifier,x_test)
    return classifier,accuracy,report,x_test,y_test,cm

def knn(x_train,y_train,x_test):
    from sklearn.neighbors import KNeighborsClassifier
    classifier=KNeighborsClassifier(n_neighbors=5,metric='minkowski',p=2)
    classifier.fit(x_train,y_train)
    classifier,accuracy,report,x_test,y_test,cm=cm_prediction(classifier,x_test)
    return classifier,accuracy,report,x_test,y_test,cm

def DecisionTree(x_train,y_train,x_test):
    from sklearn.tree import DecisionTreeClassifier
    classifier=DecisionTreeClassifier(criterion="entropy",random_state=0)
    classifier.fit(x_train,y_train)
    classifier,accuracy,report,x_test,y_test,cm=cm_prediction(classifier,x_test)
    return classifier,accuracy,report,x_test,y_test,cm

def RandomForest(x_train,y_train,x_test):
    from sklearn.ensemble import RandomForestClassifier
    classifier=RandomForestClassifier(n_estimators=10, criterion="entropy",random_state=0)
    classifier.fit(x_train,y_train)
    classifier,accuracy,report,x_test,y_test,cm=cm_prediction(classifier,x_test)
    return classifier,accuracy,report,x_test,y_test,cm

In [3]:
def selectK_classification(acclog,accsvm1,accsvmn1,accknn,accnav,accdes,accrf):
    dataframe=pd.DataFrame(index=["chiSquare"],columns=["Logistic","SVM1","SVMn1",
                                                        "KNN","Navie","Decision","Random"])
    for number,idex in enumerate(dataframe.index):
        dataframe["Logistic"][idex]=acclog[number]
        dataframe["SVM1"][idex]=accsvm1[number]
        dataframe["SVMn1"][idex]=accsvmn1[number]
        dataframe["KNN"][idex]=accknn[number]
        dataframe["Navie"][idex]=accnav[number]
        dataframe["Decision"][idex]=accdes[number]
        dataframe["Random"][idex]=accrf[number]
    return dataframe    

In [4]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df=pd.get_dummies(df2,drop_first=True)
indep_x=df.drop("classification_yes",axis=1)
dep_y=df["classification_yes"]

In [5]:
df2

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.000000,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.000000,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.000000,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.000000,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.000000,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,219.000000,...,37.000000,9800.000000,4.400000,no,no,no,yes,poor,no,yes
395,51.492308,70.000000,c,0.0,2.0,normal,normal,notpresent,notpresent,220.000000,...,27.000000,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes
396,51.492308,70.000000,c,3.0,0.0,normal,normal,notpresent,notpresent,110.000000,...,26.000000,9200.000000,3.400000,yes,yes,no,poor,poor,no,yes
397,51.492308,90.000000,a,0.0,0.0,normal,normal,notpresent,notpresent,207.000000,...,38.868902,8408.191126,4.705597,yes,yes,no,yes,poor,yes,yes


In [6]:
df

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 28 columns):
age                   399 non-null float64
bp                    399 non-null float64
al                    399 non-null float64
su                    399 non-null float64
bgr                   399 non-null float64
bu                    399 non-null float64
sc                    399 non-null float64
sod                   399 non-null float64
pot                   399 non-null float64
hrmo                  399 non-null float64
pcv                   399 non-null float64
wc                    399 non-null float64
rc                    399 non-null float64
sg_b                  399 non-null uint8
sg_c                  399 non-null uint8
sg_d                  399 non-null uint8
sg_e                  399 non-null uint8
rbc_normal            399 non-null uint8
pc_normal             399 non-null uint8
pcc_present           399 non-null uint8
ba_present            399 non-null uint8
htn_yes 

In [8]:
kbest=selectkbest(indep_x,dep_y,6)
acclog=[]
accsvm1=[]
accsvmn1=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]

In [15]:
kbest

array([[3.00000000e+00, 1.48112676e+02, 5.74821053e+01, 3.07735602e+00,
        3.88689024e+01, 8.40819113e+03],
       [2.00000000e+00, 1.48112676e+02, 2.20000000e+01, 7.00000000e-01,
        3.40000000e+01, 1.23000000e+04],
       [1.00000000e+00, 9.90000000e+01, 2.30000000e+01, 6.00000000e-01,
        3.40000000e+01, 8.40819113e+03],
       ...,
       [3.00000000e+00, 1.10000000e+02, 1.15000000e+02, 6.00000000e+00,
        2.60000000e+01, 9.20000000e+03],
       [0.00000000e+00, 2.07000000e+02, 8.00000000e+01, 6.80000000e+00,
        3.88689024e+01, 8.40819113e+03],
       [0.00000000e+00, 1.00000000e+02, 4.90000000e+01, 1.00000000e+00,
        5.30000000e+01, 8.50000000e+03]])

In [9]:
x_train,x_test,y_train,y_test=split_scalar(kbest,dep_y)

classifier,accuracy,report,x_test,y_test,cm=logistic(x_train,y_train,x_test)
acclog.append(accuracy)

classifier,accuracy,report,x_test,y_test,cm=svm_linear(x_train,y_train,x_test)
accsvm1.append(accuracy)

classifier,accuracy,report,x_test,y_test,cm=svm_NL(x_train,y_train,x_test)
accsvmn1.append(accuracy)

classifier,accuracy,report,x_test,y_test,cm=knn(x_train,y_train,x_test)
accknn.append(accuracy)

classifier,accuracy,report,x_test,y_test,cm=Navie(x_train,y_train,x_test)
accnav.append(accuracy)

classifier,accuracy,report,x_test,y_test,cm=DecisionTree(x_train,y_train,x_test)
accdes.append(accuracy)

classifier,accuracy,report,x_test,y_test,cm=RandomForest(x_train,y_train,x_test)
accrf.append(accuracy)

result=selectK_classification(acclog,accsvm1,accsvmn1,accknn,accnav,accdes,accrf)


C:\Users\saran\anaconda3\envs\AI\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\saran\anaconda3\envs\AI\lib\site-packages\sklearn\neighbors\base.py:441: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Users\saran\anaconda3\envs\AI\lib\site-packages\sklearn\neighbors\base.py:441: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  old_joblib = LooseVersion(joblib_version) < LooseVersion('0.12')
C:\Users\saran\anaconda3\envs\AI\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\saran\anaconda3\envs\AI\lib\site-packages\sklearn\utils\fixes.py:230: D

In [10]:
result
#5

,Logistic,SVM1,SVMn1,KNN,Navie,Decision,Random
chiSquare,0.96,0.96,0.96,0.93,0.89,0.97,0.97


In [11]:
result
#4

,Logistic,SVM1,SVMn1,KNN,Navie,Decision,Random
chiSquare,0.96,0.96,0.96,0.93,0.89,0.97,0.97


In [12]:
result
#3

,Logistic,SVM1,SVMn1,KNN,Navie,Decision,Random
chiSquare,0.96,0.96,0.96,0.93,0.89,0.97,0.97


In [13]:
result
#2

,Logistic,SVM1,SVMn1,KNN,Navie,Decision,Random
chiSquare,0.96,0.96,0.96,0.93,0.89,0.97,0.97


In [14]:
result
#6

,Logistic,SVM1,SVMn1,KNN,Navie,Decision,Random
chiSquare,0.96,0.96,0.96,0.93,0.89,0.97,0.97
